# Preparing the environment for running Garfield++ on Colab

This notebook shows how to install the necessary programs to create finite element field maps and import them to Garfield++ on Colab.

In [1]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


# ROOT installation

Note that here it is important to provide the correct Python executable and library so that the Python components are compiled for the Python3 version installed on Colab. This is done using the flags for example `-DPYTHON_EXECUTABLE` when running `cmake`

In [ ]:
!apt-get install libxpm-dev
!wget https://root.cern/download/root_v6.22.02.source.tar.gz
!mkdir root
!mv root_v6.22.02.source.tar.gz root
!cd root && tar -xvf root_v6.22.02.source.tar.gz
!cd root && mkdir build
!export LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu:/content/root/install/lib:$LD_LIBRARY_PATH
!cd root/build && cmake -Dpython3=ON -DPYTHON_EXECUTABLE=/usr/bin/python3 -DPYTHON_LIBRARY=/usr/lib/x86_64-linux-gnu/libpython3.6m.so -DPYTHON_INCLUDE_DIR=/usr/include/python3.6m -DCMAKE_INSTALL_PREFIX=`pwd`/../install ../root-6.22.02
!cd root/build && make -j 2
!cd root/build && make install

# To double-check ROOT configuration options:
!cd root/build && cat ROOTConfig.cmake

Confirm that this worked by loading relevant libraries and importing `ROOT`

In [3]:
import sys
sys.path.append("/content/root/install/lib")
import ctypes
ctypes.cdll.LoadLibrary('/content/root/install/lib/libCore.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libThread.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libtbb.so.2')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libImt.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libRIO.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libNet.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libTree.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libMathCore.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libMatrix.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libHist.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGeom.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libXMLIO.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGdml.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGraf.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGpad.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libGraf3d.so')
ctypes.cdll.LoadLibrary('/content/root/install/lib/libcppyy_backend3_6.so')

import ROOT

Welcome to JupyROOT 6.22/02


## Garfield++ installation

In [4]:
!git clone https://gitlab.cern.ch/garfield/garfieldpp.git
!apt-get install libgsl-dev

Cloning into 'garfieldpp'...
remote: Enumerating objects: 261, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 12187 (delta 132), reused 215 (delta 112), pack-reused 11926
Receiving objects: 100% (12187/12187), 58.19 MiB | 14.45 MiB/s, done.
Resolving deltas: 100% (8852/8852), done.


In [ ]:
#!mkdir garfield
#!mv garfieldpp garfield
!mkdir garfield/build
!export ROOTSYS="/content/root/install"
!export PATH="$ROOTSYS/bin:$PATH"
!export LD_LIBRARY_PATH="$ROOTSYS/lib:$LD_LIBRARY_PATH"
!cd garfield/build && cmake -DWITH_EXAMPLES=OFF -DPYTHON_EXECUTABLE=/usr/bin/python3 -DPYTHON_LIBRARY=/usr/lib/x86_64-linux-gnu/libpython3.6m.so -DPYTHON_INCLUDE_DIR=/usr/include/python3.6m -DCMAKE_INSTALL_PREFIX=`pwd`/../install ../garfieldpp
!cd garfield/build && make -j 2
!cd garfield/build && make install

Confirm that this worked by loading the Garfield shared library with `ROOT`. Note that `GARFIELD_HOME` must be set or there will be an error locating the HEED database.

In [29]:
import os
os.environ["GARFIELD_HOME"] = "/content/garfield/garfieldpp"
ROOT.gSystem.Load("/content/garfield/install/lib/libGarfield.so")

0

Heed:
    Database path: /content/garfield/garfieldpp/Heed/heed++/database


## Gmsh installation

Here we download the binary directly

In [30]:
!wget https://gmsh.info/bin/Linux/gmsh-4.5.6-Linux64.tgz

--2020-10-14 23:24:13--  https://gmsh.info/bin/Linux/gmsh-4.5.6-Linux64.tgz
Resolving gmsh.info (gmsh.info)... 139.165.160.20
Connecting to gmsh.info (gmsh.info)|139.165.160.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30814702 (29M) [application/x-gzip]
Saving to: ‘gmsh-4.5.6-Linux64.tgz’

gmsh-4.5.6-Linux64. 100%[===================>]  29.39M  9.26MB/s    in 3.4s    

2020-10-14 23:24:17 (8.73 MB/s) - ‘gmsh-4.5.6-Linux64.tgz’ saved [30814702/30814702]



And un-tar it

In [ ]:
!tar -xvf gmsh-4.5.6-Linux64.tgz
!mv gmsh-4.5.6-Linux64 gmsh

Check to make sure it runs

In [32]:
!./gmsh/bin/gmsh -version

4.5.6


## Elmer installation
Be sure to compile with MPI off, or the program tends to crash at the very end of execution.

In [ ]:
!apt-get install libblas-dev
!mkdir elmer
!cd elmer && git clone https://github.com/ElmerCSC/elmerfem.git
!cd elmer && mkdir build
!cd elmer/build && cmake -DWITH_MPI:BOOLEAN=FALSE -DCMAKE_INSTALL_PREFIX=`pwd`/../install ../elmerfem
!cd elmer/build && make
!cd elmer/build && make install

Try running an Elmer executable to make sure it worked

In [34]:
os.environ["PATH"] += ":/content/elmer/install/bin"
!ElmerSolver

ELMER SOLVER (v 8.4) STARTED AT: 2020/10/14 23:35:58
ParCommInit:  Initialize #PEs:            1
MAIN: 
MAIN: =============================================================
MAIN: ElmerSolver finite element software, Welcome!
MAIN: This program is free software licensed under (L)GPL
MAIN: Copyright 1st April 1995 - , CSC - IT Center for Science Ltd.
MAIN: Webpage http://www.csc.fi/elmer, Email elmeradm@csc.fi
MAIN: Version: 8.4 (Rev: 6a6d25a1, Compiled: 2020-10-14)
MAIN:  Running one task without MPI parallelization.
MAIN:  Running with just one thread per task.
MAIN: =============================================================
ERROR:: ElmerSolver: Unable to find ELMERSOLVER_STARTINFO, can not execute.
STOP 1


## Save binaries for faster setup

In [ ]:
!tar -czvf environ.tar.gz elmer/install root/install garfield/install garfield/garfieldpp gmsh/bin

In [36]:
!ls -l environ.tar.gz

-rw-r--r-- 1 root root 380469033 Oct 14 23:37 environ.tar.gz
